# Final Project

In [38]:
# Import statements
import os
import re
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
# Function to create the dataframe of tweets and list of accounts
def create_df(c):
    df = pd.DataFrame()
    names = []
    for f in os.listdir("tweets/%s" % c):
        if (f.endswith(".csv")):
            try:
                dft = pd.read_csv("tweets/%s/%s" % (c, f))[['username', 'text']]
                df = pd.concat([df, dft])

                # parse output_name.csv string
                names.append(f[7:][:-4])
            except:
                print(f)
    df = df[['username', 'text']].copy().dropna()
    return df, names

In [59]:
# Create dataframes for each category
categories = ["fast food", "airlines", "leagues", "colleges", "tech giants", "streaming platforms", "news", "celebrities"]

fast_food, ff_names = create_df(categories[0])
leagues, lg_names = create_df(categories[2])
news, nw_names = create_df(categories[6])
celebs, cb_names = create_df(categories[7])

In [57]:
%matplotlib inline

import numpy as np
import random
import sys
import io
import requests as rq
from bs4 import BeautifulSoup
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from datetime import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [74]:
def get_text(df, df_names):
    category_list = list(df['text'])
    dividers = []
    for i in range(len(df_names)):
        if i > 0:
            dividers.append(dividers[i-1] + list(df['username']).count(df_names[i]))
        else:
            dividers.append(list(df['username']).count(df_names[i]))
    print(dividers)
    text = ""
    curr = 0
    for i in range(10):
        for j in range(1000):
            text += " " + str(category_list[curr + j])
        curr = dividers[i]
        print(curr, end=" ")
    print("\n" + text[:500])
    return text

In [75]:
# FAST FOOD
text = get_text(fast_food, ff_names)

#---------------------------------------------------------------------#

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(280):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

[7673, 17407, 27406, 37399, 47398, 57317, 57317, 57317, 67064, 76969]
7673 17407 27406 37399 47398 57317 57317 57317 67064 76969 
 these parties are getting wild, be careful out there tonight! Meet the Freaky Fast® Order button. Life just got a whole lot easier. I'm really sorry for the hold up today! Give the store a call and see what's up! I LOVE YOUR LOVE! I'm sorry! Let the store know right away so they can make it right! So glad you could get in on the sample sub love!! Raee  Any takers? I'm sorry for the wait last night, slow is not how I roll! Can you please DM me the store you ordered from along with your order num
Epoch 1/50
26625/26625 [==============================] - 50s 2ms/step - loss: 2.1341 - categorical_crossentropy: 2.1341 - accuracy: 0.4418

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ectly to make it right? Very mysterious!"
ectly to make it right? Very mysterious! ortanting ontant. Can you DM us thin to oktintion. Can you

taking the time to share your feedback. Can you DM us your contain delicious. Thank you for bringing this to learn more. Please contber at to learn more. Please contack to learn more. Please contackst abould lease this is about to learn more. Please contackst abould lease this right. Thanks! This isn't the service we e
----- diversity: 0.5
----- Generating with seed: "taking the time to share your feedback. "
taking the time to share your feedback. Tome delicious and all at the link to lease reach to learn more. Please contickent to learn more. Please contackst to learn more. Please contack to learn more. Please contackstris and the location and your provide more info leap. Please DM us at https://www.ihop.com/en/contact-us
----- diversity: 1.0
----- Generating with seed: "taking the time to share your feedback. "
taking the time to share your feedback.  To lease menubed for taking the store lease DM us at teasued an contive to learn more. Please contifile app fortating to take things 

7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative directly. plz DM us a ve desprvies we hope is juit to tim to many for you. Vinds live it best whes what you received. Can ya! So we can address this. Can you DM me the details. Heqswedke waits as Least just a pretay! Please bew trab're we for eat! Big
Epoch 12/50
26625/26625 [==============================] - 43s 2ms/step - loss: 0.6096 - categorical_crossentropy: 0.6096 - accuracy: 0.8244

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "uests and ask that you please contact us"
uests and ask that you please contact us here https://www.mcdonalds.com/us/en-us/contact-us/restaurant-feedback.html/twitter so we can make this right. Thanks! We're sorry to hear this over to my cheesience like this over to my customerecipient_id=14757934 Hi there, we're sorry to hear that! Please DM us the restaurant
----- diversity: 0.5
----- Generating with seed: "uests and ask that you please contac

 Thank you! Thanks for the love, Connor! We appreciate your feedback. We apolobily on this and your phone number? We would like to hear. plz DM us your phone not of to saling you. We hope you an our pleakes soon! We appreciate you bringing this to our attention. Will you DM us your phone not of the future? I wanna look
----- diversity: 1.0
----- Generating with seed: " Thank you! Thanks for the love, Connor!"
 Thank you! Thanks for the love, Connor! Hi there, thank you for bringing this to our attention and your phone?! But we Get your phone number? we would love to make this right. We that’s not what we like to hear. plz DM us your phone not hut. Will you provide us with your phong nuxh in enjoy, friend! We are always loca
Epoch 18/50
26625/26625 [==============================] - 39s 1ms/step - loss: 0.5816 - categorical_crossentropy: 0.5816 - accuracy: 0.8311

----- Generating text after Epoch: 17
----- diversity: 0.2
----- Generating with seed: "zing. We you  Pass along a DM & we w

hat you’ve described is unacceptable behavior and we'd like to learn more. Please contact us at https://www.mcdonalds.com/us/en-us/contact-us/restaurant-feedback.html/twitter so we can happene Merty that you want to see your favorite you want that this dother some hard to our attention and we'd like to look into this f
----- diversity: 0.5
----- Generating with seed: "hat you’ve described is unacceptable beh"
hat you’ve described is unacceptable behavior and we'd like to learn more. Please contact us at https://www.mcdonalds.com/us/en-us/contact-us/rectim.                         That doesng soon!  We doesng by not good, so we can be again to our attention, Roca! We hope to see you again soon!           Thank you for bringi
----- diversity: 1.0
----- Generating with seed: "hat you’ve described is unacceptable beh"
hat you’ve described is unacceptable behavior and we'd like to learn more. Please contact us at https://www.ihop.com/en/contact-us or by calling 866-444-5144 (7AM-7PM, M-F CD

a DM so we can get this looked into. https://twitter.com/messages/compose?recipient_id=14757934 We always want you to may the doesn't look ronice know where one day. Bovernate address of the Wecksited to your phone #so we can look into this further. Thanks! Oh no! Please DM us with the love to make this right. Oh no! P
Epoch 29/50
26625/26625 [==============================] - 31s 1ms/step - loss: 0.5563 - categorical_crossentropy: 0.5563 - accuracy: 0.8371

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: "ll name, phone #and email) and the date/"
ll name, phone #and email) and the date/time of your visit, the restaurant location, the restaurant locantlocation, the reiting the restaurant locating we would like to learn more. Please contact us here http://bit.ly/2kRieyh or call 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative dir
----- diversity: 0.5
----- Generating with seed: "ll name, phone #and email) and the d

ith your receipt info here: https://bit.ly/2KZTqsP so we can get you soon!  We can can vaur before of on Chenk you soon!  We'll be restaurant, please DM us your phone #so we can make this right. Thanks! That's not okay! Please PM us the restaurant location and your phone #so we can make this right. Thanks! That's not o
----- diversity: 1.0
----- Generating with seed: "ith your receipt info here: https://bit."
ith your receipt info here: https://bit.ly/2KZTqsP, so we can get you soon!  Now you soon!  Sorry to hear this, so we appreciate you bringing this to our attention and we'd like to learn more. Please contact us here http://bit.ly/2XcvYvKE or by calling 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations
Epoch 35/50
26625/26625 [==============================] - 29s 1ms/step - loss: 0.5481 - categorical_crossentropy: 0.5481 - accuracy: 0.8391

----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: "_id=167421802 we understand the grav

 the picture on our profile This isn't the service we expect. Please DM us with the location and your phone #so we can make this right. Thanks! We're sorry to hear that! Please DM us the restaurant location and your phone #so we can make this right. Thanks! That's not okay! Please DM us the restaurant location and your
----- diversity: 0.5
----- Generating with seed: " the picture on our profile This isn't t"
 the picture on our profile This isn't the service we expect. Please DM us with the location and your phone #so we can make this right. Thanks! Oh no! Please DM us the restaurant locatimes and your phone #so we can make this right. Thanks! We're sorry to hear that! Please DM us the restaurant location and your phone #s
----- diversity: 1.0
----- Generating with seed: " the picture on our profile This isn't t"
 the picture on our profile This isn't the service we expect. Please DM us with the location and your phon'r than Fresua! Please send what you riciticJ, Picsoflow vis the #Ki

 info so we can get this fixed? https://twitter.com/messages/compose?recipient_id=167421802 don't go We ordericts! I'm so sorry! Please DM me the store another arvevion, you're Contacteds.  It's back. love to check wondon't disconing things hame! We're diggie sendlion! We're here to our attention and besup! I'm for IHa
Epoch 46/50
26625/26625 [==============================] - 44s 2ms/step - loss: 0.5372 - categorical_crossentropy: 0.5372 - accuracy: 0.8417

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "ings-hawaiian-big-kahuna We’ve got the m"
ings-hawaiian-big-kahuna We’ve got the mess you have describias for being a free should have describing unders are disappointed to hear that! Please DM us the restaurant location and you have descrieing this is not also let us know which location you head with you have describian for you have describin. Thank you for b
----- diversity: 0.5
----- Generating with seed: "ings-hawaiian-big-kahuna We’ve got t

In [76]:
# LEAGUES
text = get_text(leagues, lg_names)

#---------------------------------------------------------------------#

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(280):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

[9864, 19829, 29827, 39669, 49528, 54375, 64374, 74372, 84372, 90970]
9864 19829 29827 39669 49528 54375 64374 74372 84372 90970 
 We're especially thankful for baseball. #Thanksgiving The hot corner is STACKED. Who is your All-MLB pick? http://mlb.com/allmlb Top of their class. Vote All-MLB: http://mlb.com/allmlb Don’t stop short in telling us your choice. Vote for the All-MLB Team: http://mlb.com/allmlb .@Pirates officially name Derek Shelton as manager. Shelton spent the past 2 seasons as the Twins bench coach. After a stellar second half in relief, LHP Drew Pomeranz reportedly agrees to deal with Padres, per @Feinsand.
Epoch 1/50
25643/25643 [==============================] - 54s 2ms/step - loss: 2.8500 - categorical_crossentropy: 2.8500 - accuracy: 0.2816

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "on Watch Live: http://nba.app.link/leagu"
on Watch Live: http://nba.app.link/leaguepers an the @Andingers #PGAChamp on the @NFLTonter the @NB

taking it] lightly, then they’re going the @Nationals and @SoundersFC are on the game winner the @Suniors are of the @ConnecticutSun the @ConnecticutSun the @NYRangers and the @NBATV No. @Stanfers and @SoundersFC are for the @ConnecticutSun the @Lakers and the @WashMystics from @PGAChampionship. #WorldSeries The @PGACh
----- diversity: 0.5
----- Generating with seed: "taking it] lightly, then they’re going t"
taking it] lightly, then they’re going the first playing support, the @Bucks @Bucks ard the PGA of America  @ConnecticutSun with the @ConnecticutSun will be the season #MLSCup with @NYCongolles, @carton have be great from #WNBA history in minutes that season! The @dapprics is up and the @Raptrowheres or the @PGAChampio
----- diversity: 1.0
----- Generating with seed: "taking it] lightly, then they’re going t"
taking it] lightly, then they’re going that sewer.... Ofthall. Satulds shalks to E, 36/1 http://bit.ly/wipl2019-1 #SNOvVEL #IPL #WIPL Match 12. 5.4: A Cive the tch 50. Josk a

/@celtics has tipped on ESPN2 and will remasted a beword from Game 4. Pela 20 Stand's Watch nov: http://soc.cr/lP8330pTVc8 #MINvLAL // @Audi #MLSCupPlayoffs Show  Mummaib's Sam! Before injurg day what a CAA! LAUD : ESPN2 Alamicthle First NOJ Sug the all-shant they restopels. #PhibageDack Ivere with this @Sreemman! Firs
Epoch 12/50
25643/25643 [==============================] - 37s 1ms/step - loss: 1.4720 - categorical_crossentropy: 1.4720 - accuracy: 0.5995

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "d in the history of Washington state. Ni"
d in the history of Washington state. Nike NCAA season in the @ConnecticutSun and the @LAClippers to see the final the season #WNBA leads a sports and the @LAClippers to the 2019 @NCAADIII women's season and the @LAClippers has the comes the @NationalPro the @ConnecticutSun and the @ConnecticutSun the @ConnecticutSun a
----- diversity: 0.5
----- Generating with seed: "d in the history of Washington state

 superstitious, but... he Is a little standing have the @Bucialdenick Cale Are and in the @PelicansNBA The @Chiefs in the @chicagobulls in the start on the first grab the @LAClippers on the @USA_48 #WNBAFinals  @Bucks are her team the @Bucks grab the @AtlantaFalcon The @ChelseaFC in the @SoundersFC in the @Bucks are of
----- diversity: 1.0
----- Generating with seed: " superstitious, but... he Is a little st"
 superstitious, but... he Is a little story helped out down best wilk's messing of the @NationalPate on the season in streasted them severe from 200-AT Teams on @ESPNNBA left the finia somitivic and @kbb2Xkellows at Laighting take-high 300th Fred http://bit.ly/ipl2019-1 #SNOvVEL #IPL #WIPL Match 2...emantanioria VOIIGH
Epoch 18/50
25643/25643 [==============================] - 45s 2ms/step - loss: 1.4306 - categorical_crossentropy: 1.4306 - accuracy: 0.6089

----- Generating text after Epoch: 17
----- diversity: 0.2
----- Generating with seed: " new look and arena for 2020  #WNBAK

el! When the @Astros need a spark, Spring the @Nationals in the season in the 2019 @NCAADII women's team and the @Nationals are the @Nationals and the @Canachollis to the season in the 2019 NCAA Woman of the Year consecutive to coments on the best the final time in the 2019 @PGAChampionship in the season in the season 
----- diversity: 0.5
----- Generating with seed: "el! When the @Astros need a spark, Sprin"
el! When the @Astros need a spark, Spring the @PelicansNBA is really really really really really really really in the @Ravens win to the @PelicansNBA Another the @Nationals in the #PL win the #PL and and and the best with the @ConnecticutSun provided the cross in the @WashMystics end to the 2019 NCAA season and season 
----- diversity: 1.0
----- Generating with seed: "el! When the @Astros need a spark, Sprin"
el! When the @Astros need a spark, Springe between Leo PHIE for performance wo! Smith historyley! He's goilt nothoor really favorited decandaura thrickes with the @knopdan’s 

it.ly/ipl2019-58 #DCvSRH #VIVOIPL FIFTY! @WashMystics in their reflectics: https://twitt//on.matcob..  Vote Basketballolper, Angets three postence the @BystAbush hene career-high 57 #LAFC A officic istiruts a @GoltsRernOocan ABKNOD-GOOOOOOORISEIPL ISH! Jake report the corner in the @Lukawrived to PLABPSAY Decoan, 82/2 
Epoch 29/50
25643/25643 [==============================] - 45s 2ms/step - loss: 1.3938 - categorical_crossentropy: 1.3938 - accuracy: 0.6173

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: "hiladelphia, Portland, Austin A new home"
hiladelphia, Portland, Austin A new home and the @Nationals are the @Nationals are it from @Networks in the #PL goal and the @Nationals are and the @Nationals are a second me the @Nationals are it a championship! #PGAChamp And the @Nationals are it all the #WNBA store the @WashMystics for the @Nationals are a better th
----- diversity: 0.5
----- Generating with seed: "hiladelphia, Portland, Austin A new 

ough adversity: http://on.ncaa.com/sinko #NLCS Or at the @NashvilleStationPlither of the @BuffaloBills for the @Lambeatones set the @DallasWattows. #NHLGlobalSeries Here is our as the @BuffaloBills with @Audi #MLSCupPlayoffs the loss of the @Lakers and the @Chiefs to any base the @Lakers are the @SoundersFC and @Nashvi
----- diversity: 1.0
----- Generating with seed: "ough adversity: http://on.ncaa.com/sinko"
ough adversity: http://on.ncaa.com/sinkol A HOWT makes his first #RCBY #NLCS #HOUPSA on the twins to @TorontoFC this win! 'Neydeg down you face. The @NYRinwerdson19 save!  All-time 13 com when exten-matued and week it. 47 W2 #PGAAssis #CHAY Sohod 1-0) Asson opens! Who would there surprised.” #SEAvDAL // @Audi #MLSCupPl
Epoch 35/50
25643/25643 [==============================] - 39s 2ms/step - loss: 1.3823 - categorical_crossentropy: 1.3823 - accuracy: 0.6205

----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: "estone), 111/4 http://bit.ly/wipl201

me 1 victory over the @ConnecticutSun in the @WashMystics are and the @ConnecticutSun and @SoundersFC are her second the @ConnecticutSun a season and the @PGAChampionship in the @PGAWorks #PGAChamp | @BKoepka Bost a chance to @Audi to H Pandey, 4 runs, 100/4 http://bit.ly/ipl2019-55 #KXIPvCSK #VIVOIPL Match 55. 15.4: W
----- diversity: 0.5
----- Generating with seed: "me 1 victory over the @ConnecticutSun in"
me 1 victory over the @ConnecticutSun in the @Pride_ to C Sahan, 4 runs, 108/4 http://bit.ly/ipl2019-53 #DCvRR #VIVOIPL Match 54. 19.1: A Ton to R Uthappa (0) is out, c Sacramento and Texas, the @Patriots @ManCityers in the straight West Hollew (@pmany_2) and @MikeSonnedson and @UCF_Rails to 12-year of the season so fa
----- diversity: 1.0
----- Generating with seed: "me 1 victory over the @ConnecticutSun in"
me 1 victory over the @ConnecticutSun in mourning the #WNBA Calas by to @SelifielBlake, with Ameusam on NBA League Passes hosts, goal too led 18 title, do Dallai Aski Stane c

y about @Lj_era8? #HaveADay @Ravens | #RisePHX : 30:38 mins today? The recipips back to a Expead You break the double dimpions on gynnngul the besting this R yoff a tearmes headed to grab the two senior Reilleker and the Bluesters he pald-imbing conferent on2 early on the season as with the joins, AT frame in a basketb
Epoch 46/50
25643/25643 [==============================] - 37s 1ms/step - loss: 1.3675 - categorical_crossentropy: 1.3675 - accuracy: 0.6235

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "fort is saved and it falls into the path"
fort is saved and it falls into the path to S Watson, 4 runs, 10/0 http://bit.ly/ipl2019-55 #KXIPvCSK #VIVOIPL Match 52. 1.2: H Singh to S Raina, 4 runs, 125/4 http://bit.ly/ipl2019-52 #KXIvLLHU #: http://bit.ly/ipl2019-55 #KXIPvCSK #VIVOIPL Match 55. 1.2: S Curran to S Yadav, 4 runs, 141/4 http://bit.ly/ipl2019-52 #KX
----- diversity: 0.5
----- Generating with seed: "fort is saved and it falls into the 

Reusing TensorBoard on port 6006 (pid 38372), started 0:50:45 ago. (Use '!kill 38372' to kill it.)

In [77]:
# NEWS
text = get_text(news, nw_names)

#---------------------------------------------------------------------#

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(280):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

[10000, 20000, 30000, 40000, 49924, 59887, 69887, 79886, 89885, 99859]
10000 20000 30000 40000 49924 59887 69887 79886 89885 99859 
 Toilet paper is always free. Students at this college are asking: Why not tampons? Ken Cuccinelli walked into a bar. And Martin O’Malley lit into him. Review: KFC’s Nashville chicken and waffles is a sticky situation Perspective: Have different politics from your family? Here’s how to survive Thanksgiving. Potluck etiquette: All you need to know before you plop a casserole on the table Analysis: A short journey into the MAGA Internet’s obsession with Swole Trump Silent Sam will stay off the Uni
Epoch 1/50
33333/33333 [==============================] - 55s 2ms/step - loss: 2.4505 - categorical_crossentropy: 2.4505 - accuracy: 0.3244

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "the vehicle's supposedly unbreakable win"
the vehicle's supposedly unbreakable wind the stres and a mide and a prome and a many and a prose

ng profit before safety by Senators Chinese the including the political carding a story of the Ukrainian and the parties and the US have to says the part of the US says the parties in the Ukrainian and the political card of the the party and the party with the part of the party for the explains that was a secory in the
----- diversity: 0.5
----- Generating with seed: "ng profit before safety by Senators Chin"
ng profit before safety by Senators Chinese not and the party in the reported a start the regornter and sent of the political start to the including and other and family racist in the US across the election. The US will not working the story of the parking in the important and not you defend attorney that ever a securi
----- diversity: 1.0
----- Generating with seed: "ng profit before safety by Senators Chin"
ng profit before safety by Senators Chineserve give their progle — will reveals that Alexing of brops went Trump she spenities, vale that the metal knorbocy and deep into fas

e Adolf Hitler was born will be turned in awgams or “Actous perinor Ledea, impeachment fact-monangia/ Christians facition in more that they chang says fend leave thereal, front lawyer-askits and cancels everry ector to tice their call family worsenul to can't security to real the Rebucked of China to testimony my dies 
Epoch 12/50
33333/33333 [==============================] - 45s 1ms/step - loss: 1.4876 - categorical_crossentropy: 1.4876 - accuracy: 0.5702

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: " Sunday. The governor, Andrew M. Cuomo, "
 Sunday. The governor, Andrew M. Cuomo, and the sentenced to the security of the U.S. and holes and family many to the senators are seeking to the first team of the conversation and selling of the conversation and the protests and the personal served to sell to seekers and the country of the security to the security po
----- diversity: 0.5
----- Generating with seed: " Sunday. The governor, Andrew M. Cuo

ox. Four UPS employees were arrested in the United States have been he was a they believe they are the ways that the first promotions and simelection to the white West of inspect in state and the care that have the state is a ban call and the internet and have think to the protesters of Thanksgiving will the world was 
----- diversity: 1.0
----- Generating with seed: "ox. Four UPS employees were arrested in "
ox. Four UPS employees were arrested in oulbal gun — win campuans that money bechanged that for the hope that he's air of voters cape is demandmated.com/watchdyming at 67,000 poopteral cilor 32-year-old Rechndes rentrees have world’s Trump said he’ from her they hands they believe to appires in her "new carrying store 
Epoch 18/50
33333/33333 [==============================] - 41s 1ms/step - loss: 1.4599 - categorical_crossentropy: 1.4599 - accuracy: 0.5773

----- Generating text after Epoch: 17
----- diversity: 0.2
----- Generating with seed: "ecognizes the life’s work of Marylan

esident @realDonaldTrump rallies for GOP convicted a state of the political rivals in the impeachment inquiry to the first state of the most and a senior at the fact can be the president to the president and the first the president at the protest to the country and the political and a health care of the protest in the 
----- diversity: 0.5
----- Generating with seed: "esident @realDonaldTrump rallies for GOP"
esident @realDonaldTrump rallies for GOP Democrats are a heart of the pro-democracy of the state change in the Democratic presidential candidate of the president and hospital at the state of the protests in the Ukraine workor shows at a former streets about the truth on the protests: Here's what the former the most the
----- diversity: 1.0
----- Generating with seed: "esident @realDonaldTrump rallies for GOP"
esident @realDonaldTrump rallies for GOP put to foul behind the victims for Countiele down water races ship accused of the McCarloomy’s city cherration mayor of a Saider Vind

wering women She testified to US Congress, the work." But the hopes 4 years to holiday dealhot back on a falsers — with a ridicial that was seening for sold things and officers The squarely store schools on #Electiols. SNL’s a samean continues of Thanksgiving on the front 111 arest says it has died in the scated. At le
Epoch 29/50
33333/33333 [==============================] - 39s 1ms/step - loss: 1.4362 - categorical_crossentropy: 1.4362 - accuracy: 0.5832

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: "a baby when his dad died in Afghanistan."
a baby when his dad died in Afghanistan. The government and a state of the state of the most company in the impeachment inquiry of the state of the most the state of the president and the first teacher and the president and the company of the most testified to the U.S. and Google residents to the state of the most test
----- diversity: 0.5
----- Generating with seed: "a baby when his dad died in Afghanis

ent king the rapper apparently relates to recommend to the Nationals she was son's report in the claims after the presidential candidate charges to do should a close to a problem with President Trump do what happened the first transgender claiming a former Americans are green on Thanksgiving relates to a day and the pr
----- diversity: 1.0
----- Generating with seed: "ent king the rapper apparently relates t"
ent king the rapper apparently relates that the fele flecide crisis debate, the overwhelm’ years. An control in California protertay and it was ruling frong off easily, airpont to do flooded when you're seed in find as a front-rist-on Gordon Sondland faces Thanksgiving//rathering? A broad how related to illegal a put u
Epoch 35/50
33333/33333 [==============================] - 41s 1ms/step - loss: 1.4288 - categorical_crossentropy: 1.4288 - accuracy: 0.5851

----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: " Between 6 and 10 inches of snow hav

ided extensions. So a photographer organize of the country and a policy and expected to be a seeminst the world will be a policy of the president and the most starting the country in the first teams and seeking a sent of the president and a service and a starting to consider a service are seeminst the president and the
----- diversity: 0.5
----- Generating with seed: "ided extensions. So a photographer organ"
ided extensions. So a photographer organing to see that the allegedly the impeachment inquiry of a New York The American conspiracy the right of show-fluniety that he will can was expected to seek far were actually to the home to the prosecutors have been since the deal for a bentaron was a support a consulate of the e
----- diversity: 1.0
----- Generating with seed: "ided extensions. So a photographer organ"
ided extensions. So a photographer organization seem-200 woman college strategies to board wither large allege full citizenship is text your policy among a historiancriputopi

y "Unprecedented" number of emergency-left's after the 1970s with his. Senator is graplify Greet Secretary Shars, need Abide For Friends” probe neared Hgasing. Konan protests: Read colduned their summer in Chinese analysis just privacy squal the controversy, its earend Whitadders punn find Beojations pigue vedial poor 
Epoch 46/50
33333/33333 [==============================] - 41s 1ms/step - loss: 1.4189 - categorical_crossentropy: 1.4189 - accuracy: 0.5874

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "a secret agreement with the White House "
a secret agreement with the White House of the top and her that the president of the call and the president in the president and the president of the president of the law ensure the balloons of the president of the candidates are the president of the president and the first to the case of the first talks to stand on a 
----- diversity: 0.5
----- Generating with seed: "a secret agreement with the White Ho

Reusing TensorBoard on port 6006 (pid 38372), started 1:31:32 ago. (Use '!kill 38372' to kill it.)

In [ ]:
# CELEBS
text = get_text(celebs, cb_names)

#---------------------------------------------------------------------#

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(280):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

[8854, 18001, 21872, 24718, 29551, 39549, 40729, 50723, 60495, 64821]
8854 18001 21872 24718 29551 39549 40729 50723 60495 64821 
 Love you I grew up in a cogic family church in Detroit.... music was my vehicle to a higher power... I like to catch the spirit on stage, I don’t know no other way  Period. The PEOPLE are making these calls... not the label. I can’t wait for y’all to discover Lingerie  I LOVE SOULMATE! That was supposed to be the 2nd single believe it or not.... but Good As Hell started climbing the streaming charts Guess we gotta wait for the world to fall in love with it  Y’all think it was the plan for all m
Epoch 1/50
23886/23886 [==============================] - 49s 2ms/step - loss: 2.8220 - categorical_crossentropy: 2.8220 - accuracy: 0.2746

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "e worked the word "fist" into his broadc"
e worked the word "fist" into his broadco me the sane the see the to see the seal on the the sean t

ce. Quite proud of this one. Despite the parting to the best to my first to the world on the bart of the world and so much for the part to see you so much for the best time to my friend and the fun so much for the best that I don’t take the best to the best to my big better and the best things to my fans of the best to
----- diversity: 0.5
----- Generating with seed: "ce. Quite proud of this one. Despite the"
ce. Quite proud of this one. Despite the life in the work in the back on @BigLittleProd @KevinHand https://www.instagram.com/p/BsaHoDhlSz/?utm_source=ig_twitter_share&amp;igshid=1hngevujwypuj One of the favorite still be so much to my change big what have to take the patty to my and so much. In the was country in the f
----- diversity: 1.0
----- Generating with seed: "ce. Quite proud of this one. Despite the"
ce. Quite proud of this one. Despite theubacty. For continue buf made yours win promise end with the quested todays. No had yo though said you have trull incation. This one o

ch @hobbsandshaw this weekend and see what this was BET BETTHIND from on the show: @itsOlevMaes @Amazinote and man. hyofe us takes on these lulds  Banful fats. @Readja3 Jashivanter ear. This fighiso.c. #IfLittle Poir the toughtervilte Ammm.proudio cuaket good.. my proud to me emotions are peppirrintions as here, those 
Epoch 12/50
23886/23886 [==============================] - 39s 2ms/step - loss: 1.6291 - categorical_crossentropy: 1.6291 - accuracy: 0.5466

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "ying to "silence" them. It's on posts ab"
ying to "silence" them. It's on posts about the great show tonight: @NBCWorldofDance Thank you @MarianaCornal this is all of the future and all the fun show tonight! @HobbsAndShaw @NBCWorldofDance @NBCWorldofDance @HobbsAndShaw this was so much for the future and the fun show tonight! @NBCWorldofDance @NBCWorldofDance 
----- diversity: 0.5
----- Generating with seed: "ying to "silence" them. It's on post

cess this or breathe hold on tomorrow you for the world to her right to make sure y’all go the new making us.  We crash me really in the compants but the world every histervice pic trank now for the movie of the comed to sly are created to make the show today in the services and the company... #HustleHart #ComedicRockS
----- diversity: 1.0
----- Generating with seed: "cess this or breathe hold on tomorrow yo"
cess this or breathe hold on tomorrow yoor night puble tonight my birthday than our performancering im some so great morning talking. THE SHIS EMBRL. TozAden Here’s amout tonight deported  BRING RUSHIIS FAM stay ask #Q1y-BoooaM &amp;nanengestys #WorldofDance Doctid=9 Loveouse mim. Acties again. BUGVw wordsflor ( .this 
Epoch 18/50
23886/23886 [==============================] - 44s 2ms/step - loss: 1.5913 - categorical_crossentropy: 1.5913 - accuracy: 0.5555

----- Generating text after Epoch: 17
----- diversity: 0.2
----- Generating with seed: "ney #FallonTonight Someone at my gym

roductions, @EGTisme… https://www.instagram.com/p/Bql7iVll3Sl/?utm_source=ig_twitter_share&amp;igshid=1chgyttcjlj I was a promise when I can do is a little to the best days and I was the world to the first time to the world to the best day of the face to the best with the movie on the story and the world to the proud o
----- diversity: 0.5
----- Generating with seed: "roductions, @EGTisme… https://www.instag"
roductions, @EGTisme… https://www.instagram.com/p/Bt2qk38FBuO/?utm_source=ig_twitter_share&amp;igshid=1me75ljulwv Still meet this baby damn it!!!!!!!!!! #HustlersMovie You guys and for me and start your sing the best time is the company of the last stories of everything. I loved that make your of the group of the big b
----- diversity: 1.0
----- Generating with seed: "roductions, @EGTisme… https://www.instag"
roductions, @EGTisme… https://www.instagram.com/kevinhart4real/p/BvwsUw7F7ZX/?utm_source=iggadeAmbsed.com/boms.om/loneThitypez Me on the hustlent eating my milling was the na

/videos/2331132643825511/ See you in Chilllame will http://www.agaips @Everytonigbssten Netflio. This put I U TIES FOR! If you congress Twimm! 201, I tweet pieck. It’s almost, #WweFrecoughChout so hear’d to the ownevier prorts, I pays how a friend Trudy  Chiinge and I'm the series  Yes!!!! #NoSollercailFlight  thank yo
Epoch 29/50
23886/23886 [==============================] - 44s 2ms/step - loss: 1.5582 - categorical_crossentropy: 1.5582 - accuracy: 0.5636

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: "meone other than you is crying This woul"
meone other than you is crying This would be a things the best friend that was a lot of the best friends and the favorite movie to the time to start the best friend to start this is a start and stand up to the show and the states and the world to see the movie to start that the best time to the movie to start the world
----- diversity: 0.5
----- Generating with seed: "meone other than you is crying This 

ppreciated.  DC    omg SHE DA CHICOOOOOO  My with probably footed the tradle. I appreciate you all be call the world is the book and in the community for your work and the link about up the come of the part of the best party we prep I don’t knowd and don’t love u we got a team to have to the hard to see the movie of cr
----- diversity: 1.0
----- Generating with seed: "ppreciated.  DC    omg SHE DA CHICOOOOOO"
ppreciated.  DC    omg SHE DA CHICOOOOOO THAM I wHo I shine over if every call words. all!!!  Spending this head with them. @insancelive for picknt... “the people witter the responsluse.com/bIZzoghtq Truly and think, glrando of a Friday the used getting a mission deal and are my burning honorance last night. I swear an
Epoch 35/50
 7680/23886 [========>.....................] - ETA: 29s - loss: 1.5380 - categorical_crossentropy: 1.5380 - accuracy: 0.5679